In [1]:
from new_module_sdk import Workspace, DatasetX, ModuleStepX, ExperimentX
from azureml.studio.tools.module_repo import ModuleRepo

### Get workspace

In [2]:
ws = Workspace.from_config()

### Register module

In [3]:
ModuleRepo.register(workspace=ws, module_source="https://github.com/hirofumi-s-friends/recommenders/tree/cat_demo")

### Get dataset

In [3]:
dataset = DatasetX.get_by_data_reference(ws, "GenericCSV/Movie_Ratings")

### Get and define modules

In [5]:
stratified_splitter = ModuleStepX.get(ws, name='Stratified Splitter')
stratified_splitter.inputs.input_path = dataset
stratified_splitter.params.user_column = 'UserId'
stratified_splitter.params.item_column = 'MovieId'
stratified_splitter.params.seed = 42
stratified_splitter.params.ratio = 0.8

Input: ['Input_path']
Mapping property to field names.
self.input_path => self['Input_path']

Output: ['Output_train_data', 'Output_test_data']
Mapping property to field names.
self.output_train_data => self['Output_train_data']
self.output_test_data => self['Output_test_data']

Parameter: ['Ratio', 'User column', 'Item column', 'Seed']
Mapping property to field names.
self.ratio => self['Ratio']
self.user_column => self['User column']
self.item_column => self['Item column']
self.seed => self['Seed']



In [7]:
sar_train = ModuleStepX.get(ws, name='SAR Train')
sar_train.inputs.input_path = stratified_splitter.outputs.output_train_data
sar_train.params.user_column = 'UserId'
sar_train.params.item_column = 'MovieId'
sar_train.params.rating_column = 'Rating'
sar_train.params.timestamp_column = 'Timestamp'

Input: ['Input_path']
Mapping property to field names.
self.input_path => self['Input_path']

Output: ['Output_model']
Mapping property to field names.
self.output_model => self['Output_model']

Parameter: ['User column', 'Item column', 'Rating column', 'Timestamp column', 'Normalize', 'Time decay']
Mapping property to field names.
self.user_column => self['User column']
self.item_column => self['Item column']
self.rating_column => self['Rating column']
self.timestamp_column => self['Timestamp column']
self.normalize => self['Normalize']
self.time_decay => self['Time decay']



In [8]:
sar_score = ModuleStepX.get(ws, name='SAR Score')
sar_score.inputs.trained_model = sar_train.outputs.output_model
sar_score.inputs.dataset_to_score = stratified_splitter.outputs.output_test_data
sar_score.params.score_type = 'Rating prediction'
sar_score.params.items_to_predict = 'Items in score set'

Input: ['Trained_model', 'Dataset_to_score']
Mapping property to field names.
self.trained_model => self['Trained_model']
self.dataset_to_score => self['Dataset_to_score']

Output: ['Score_result']
Mapping property to field names.
self.score_result => self['Score_result']

Parameter: ['Score type', 'Items to predict', 'Ranking metric', 'Remove seen items', 'Top k', 'Sort top k', 'Normalize']
Mapping property to field names.
self.score_type => self['Score type']
self.items_to_predict => self['Items to predict']
self.ranking_metric => self['Ranking metric']
self.remove_seen_items => self['Remove seen items']
self.top_k => self['Top k']
self.sort_top_k => self['Sort top k']
self.normalize => self['Normalize']



### Submit pipeline

In [9]:
steps = [stratified_splitter, sar_train, sar_score]
ExperimentX.submit(ws, steps, 'test_experiment_sar')

ModuleStep Stratified Splitter
Inputs:  {'Input_path': $AZUREML_DATAREFERENCE_Dataset}
Outputs:  {'Output_test_data': $AZUREML_DATAREFERENCE_0bfc061d372a4078b2cb1154b4013c66, 'Output_train_data': $AZUREML_DATAREFERENCE_f41dcb7d3e1f4c85b311e348979d300d}
Parameters:  {'User column': 'UserId', 'Item column': 'MovieId', 'Seed': 42, 'Ratio': 0.8}


ModuleStep SAR Train
Inputs:  {'Input_path': $AZUREML_DATAREFERENCE_f41dcb7d3e1f4c85b311e348979d300d}
Outputs:  {'Output_model': $AZUREML_DATAREFERENCE_870be0b2bb5b446eb7cfc57dd2b7e659}
Parameters:  {'User column': 'UserId', 'Item column': 'MovieId', 'Rating column': 'Rating', 'Timestamp column': 'Timestamp'}


ModuleStep SAR Score
Inputs:  {'Trained_model': $AZUREML_DATAREFERENCE_870be0b2bb5b446eb7cfc57dd2b7e659, 'Dataset_to_score': $AZUREML_DATAREFERENCE_0bfc061d372a4078b2cb1154b4013c66}
Outputs:  {'Score_result': $AZUREML_DATAREFERENCE_ea2a060b4a8d4c278f7bd0b498fb3537}
Parameters:  {'Score type': 'Rating prediction', 'Items to predict': 'Items